In [ ]:
import pandas as pd
from bs4 import BeautifulSoup
import requests
import csv
import time

In [ ]:
url = "https://www.googleapis.com/books/v1/volumes"
api_key = "YOUR_API_KEY"  # removed for security
genres = ["Mystery", "Romance", "History", "Science", "Fantasy", "Technology", "Art", "Music", "Business", "Health"]
special_query = "harry potter"
books_per_request = 40
max_books = 5000
books = []

In [ ]:
for genre in genres:
    for start in range(0, 200, books_per_request):
        print(f"Genre: {genre}, Books {start + 1} to {start + books_per_request}...")

        params = {
            "q": f"subject:{genre}",
            "startIndex": start,
            "maxResults": books_per_request,
            "langRestrict": "en",
            "key": API_KEY
        }

        response = requests.get(url, params=params)
        if response.status_code != 200:
            print(f"Error {response.status_code} at startIndex {start}")
            continue

        data = response.json()
        for item in data.get("items", []):
            info = item.get("volumeInfo", {})
            image_links = info.get("imageLinks", {})

            # Basic required fields: title and authors only
            if "title" not in info or "authors" not in info:
                continue

            book = {
                "title": info.get("title", "").strip(),
                "authors": ", ".join(info.get("authors", [])).strip(),
                "description": info.get("description", ""),
                "category": ", ".join(info.get("categories", [])),
                "averageRating": info.get("averageRating", ""),
                "ratingsCount": info.get("ratingsCount", ""),
                "thumbnail": image_links.get("thumbnail", "")
            }

            books.append(book)

            if len(books) >= max_books:
                break

        if len(books) >= max_books:
            break

        time.sleep(1)

    if len(books) >= max_books:
        break

print("Adding Harry Potter books...")
for start in range(0, 40, books_per_request):
    params = {
        "q": special_query,
        "startIndex": start,
        "maxResults": books_per_request,
        "langRestrict": "en",
        "key": API_KEY
    }

    response = requests.get(url, params=params)
    if response.status_code != 200:
        continue

    data = response.json()
    for item in data.get("items", []):
        info = item.get("volumeInfo", {})
        image_links = info.get("imageLinks", {})

        if "title" not in info or "authors" not in info:
            continue

        book = {
            "title": info.get("title", "").strip(),
            "authors": ", ".join(info.get("authors", [])).strip(),
            "description": info.get("description", ""),
            "category": ", ".join(info.get("categories", [])),
            "averageRating": info.get("averageRating", ""),
            "ratingsCount": info.get("ratingsCount", ""),
            "thumbnail": image_links.get("thumbnail", "")
        }

        books.append(book)

    time.sleep(1)

df = pd.DataFrame(books)
df.drop_duplicates(subset=["title", "authors"], inplace=True)
df.reset_index(drop=True, inplace=True)

print(f"\n Total books collected (including incomplete & Harry Potter): {len(df)}")

In [ ]:
df

In [ ]:
# Export to CSV
df.to_csv("google_books_data.csv", index=False, encoding="utf-8-sig")
print("✅ Data saved to google_books_data.csv")

In [ ]:
df.isnull().sum()